# Modified Dresher Procedure


(i) Set t := 0, and let $Φ^0_2$ := $Φ_2$.

(ii) Find an optimal solution to $P_t$.

(iii) Find an optimal solution to $Q_t$. Let $Φ^{t+1}_2$ be those $k \in \phi_2^t$ where $p_k$ =1 in the optimal solution found.

(iv) If $Φ^{t+1}_2 = ∅$ then go to (v) else replace t by t+1 and go to (ii).

(v) The set of D-optimal strategies of Player 1 in Γ is the set of optimal solutions to $P_t$. Output any one of these optimal solutions.


In [768]:
from amplpy import AMPL
import pandas as pd

## Test (Parsimonious penny matching)

In [769]:
def P_0(A_T, I):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        
        # Params
        param A_T {I_2, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        
        # Constraints
        subject to c2{i in I_2}: sum{j in I_1} A_T[i, j] * x[j] >= z;
        subject to c3: sum{j in I_1} x[j] = 1;
        
        # Objective
        maximize obj: z;
        """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    
    ampl.param['A_T'] = A_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    x = ampl.getVariable('x').getValues().toPandas()
    z = ampl.getObjective('obj').value()
    
    # close AMPL
    ampl.close()
    
    return z,x

In [770]:
def Q_0(A_pp_T, I):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        
        # Params
        param t;
        param A_pp_T {I_2, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        var p{I_2} >= 0, <= 1;
        
        # Constraints
        subject to c2{i in I_2}: sum{j in I_1} A_pp_T[i, j] * x[j] >= p[i];
        
        # Objective
        maximize obj: sum{i in I_2} p[i];
        """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    
    ampl.param['A_pp_T'] = A_pp_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    x = ampl.getVariable('x').getValues().toPandas()
    p = ampl.getVariable('p').getValues().toPandas()
    
    # close AMPL
    ampl.close()
    
    return p,x

In [771]:
def P_1(A_T, A_p_T, I, I_2_p):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        set I_2_p;
        
        # Params
        param A_T {I_2, I_1};
        param A_p_T {I_2_p, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        
        # Constraints
        subject to c1{i in I_2_p}: sum{j in I_1} A_p_T[i, j] * x[j] >= 0;
        subject to c2{i in I_2}: sum{j in I_1} A_T[i, j] * x[j] >= z;
        subject to c3: sum{j in I_1} x[j] = 1;
        
        # Objective
        maximize obj: z;
        """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    ampl.set['I_2_p'] = I_2_p
    
    ampl.param['A_T'] = A_T
    ampl.param['A_p_T'] = A_p_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    x = ampl.getVariable('x').getValues().toPandas()
    z = ampl.getObjective('obj').value()
    
    # close AMPL
    ampl.close()
    
    return z,x

In [772]:
def Q_1(A_p_T, A_pp_T, I, I_2_p):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        set I_2_p;
        
        # Params
        param t;
        param A_pp_T {I_2, I_1};
        param A_p_T {I_2_p, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        var p{I_2} >= 0, <= 1;
        
        # Constraints
        subject to c1{i in I_2_p}: sum{j in I_1} A_p_T[i, j] * x[j] >= 0;
        subject to c2{i in I_2}: sum{j in I_1} A_pp_T[i, j] * x[j] >= p[i];
        
        # Objective
        maximize obj: sum{i in I_2} p[i];
        """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    ampl.set['I_2_p'] = I_2_p
    
    ampl.param['A_pp_T'] = A_pp_T
    ampl.param['A_p_T'] = A_p_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    x = ampl.getVariable('x').getValues().toPandas()
    p = ampl.getVariable('p').getValues().toPandas()
    
    # close AMPL
    ampl.close()
    
    return p,x

In [773]:
# Game
Players = [1, 2]

A_1 = ["guess_heads_up", "guess_tails_up"]
A_2 = ["hide penny heads up", "hide penny tails up", "tease"]
I = {1: A_1, 2: A_2}

U1 = [[1,0,0], 
    [0,1,0]]

U2 = [[0,1,1], 
    [1,0,1]]

U = {1: U1, 2: U2}

In [774]:
# Algorithm
# --------Iteration 0--------
t = 0;

U_df = {player: pd.DataFrame(U[player], index=I[1], columns=I[2]) for player in Players}
A_T = U_df[1].transpose()

z, x = P_0(A_T, I)

A_pp_T = A_T - z

p, x = Q_0(A_pp_T, I)

# Put in A_prime the actions that are not superflous, the 
A_p_T = A_T.loc[p.iloc[:,0] == 0] 

print(A_p_T)  

# Leave only superflous actions
A_T = A_T.loc[p.iloc[:,0] != 0]
print(A_T)
# I[2] = [I[2][i] for i in range(len(I[2])) if p.iloc[i,0] != 0]

# --------Iteration 1--------
t = 1

I_2_p = [I[2][i] for i in range(len(I[2])) if p.iloc[i,0] == 0]
I[2] = [I[2][i] for i in range(len(I[2])) if p.iloc[i,0] != 0]

z, x = P_1(A_T, A_p_T, I, I_2_p)

print(f"\nResults P_{1}:")
print("z:", z)
print("x:\n",x.iloc[:,0])

A_pp_T = A_T - z

p, x = Q_1(A_p_T, A_pp_T, I, I_2_p)

print(f"\nResults Q_{1}:")
print("p:", p)
print("x:\n",x.iloc[:,0])


while True:
    break
    T = range(t)
    
    z, x = P(T, A_T, A_prime_T, I)

    print(f"\nResults P_{t}:")
    print("z:", z)
    print("x:\n",x.iloc[:,0])
    
    p, x = Q_0(A_T, I)
    
    print(f"\nResults Q_{t}:")
    print("p:", p)
    print("x:\n",x.iloc[:,0])
    
    t+=1
    break
    # Break condition
    if all(p.iloc[:,0] == 0):
        break

Gurobi 9.5.1: optimal solution; objective 0
Gurobi 9.5.1: 

optimal solution; objective 2
       guess_heads_up  guess_tails_up
tease               0               0
                     guess_heads_up  guess_tails_up
hide penny heads up               1               0
hide penny tails up               0               1
Gurobi 9.5.1: optimal solution; objective 0.5
1 simplex iterations

Results P_1:
z: 0.5
x:
 guess_heads_up    0.5
guess_tails_up    0.5
Name: x.val, dtype: float64
Gurobi 9.5.1: optimal solution; objective 0

Results Q_1:
p:                      p.val
hide penny heads up      0
hide penny tails up      0
x:
 guess_heads_up    0
guess_tails_up    0
Name: x.val, dtype: int64


# General function with procedure

In [775]:
def P_legacy(T, A_T, A_prime_T, I):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        set T;
        
        # Params
        param A_T {I_2, I_1};
        param A_prime_T {T, I_2, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        
        # Constraints
        
        subject to c1{t in T, i in I_2}: sum{j in I_1} A_prime_T[t,i,j] * x[j] >= 0;
        
        subject to c2{i in I_2}: sum{j in I_1} A_T[i, j] * x[j] >= z;
        
        subject to c3: sum{j in I_1} x[j] = 1;
        
        # Objective
        
        maximize obj: z;
        """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    ampl.set['T'] = T
    
    ampl.param['A_T'] = A_T
    ampl.param['A_prime_T'] = A_prime_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    x = ampl.getVariable('x').getValues().toPandas()
    z = ampl.getObjective('obj').value()
    
    # close AMPL
    ampl.close()
    
    return z,x

In [776]:
def P(T, A_T, A_p_T, I, I_2_p):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        set I_2_p;
        set T;
        
        # Params
        param A_T {I_2, I_1};
        param A_p_T {T, I_2_p, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        
        # Constraints
        subject to c1{t in T, i in I_2_p}: sum{j in I_1} A_p_T[t, i, j] * x[j] >= 0;
        subject to c2{i in I_2}: sum{j in I_1} A_T[i, j] * x[j] >= z;
        subject to c3: sum{j in I_1} x[j] = 1;
        
        # Objective
        maximize obj: z;
    """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    ampl.set['I_2_p'] = I_2_p
    ampl.set['T'] = T
    
    ampl.param['A_T'] = A_T
    ampl.param['A_p_T'] = A_p_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    x = ampl.getVariable('x').getValues().toPandas()
    z = ampl.getObjective('obj').value()
    
    ampl.close()
    
    return z,x

In [777]:
def Q(T, A_p_T, A_pp_T, I, I_2_p):
    ampl = AMPL()
    
    ampl.eval(r"""
        # Sets
        set I_1;
        set I_2;
        set I_2_p;
        set T;
        
        # Params
        param A_p_T {T, I_2_p, I_1};
        param A_pp_T {I_2, I_1};
        
        # Vars
        var z;
        var x{I_1} >= 0;
        var p{I_2} >= 0, <= 1;
        
        # Constraints
        subject to c1{t in T, i in I_2_p}: sum {j in I_1} A_p_T[t, i, j] * x[j] >= 0;
        subject to c2{i in I_2}: sum{j in I_1} A_pp_T[i, j] * x[j] >= p[i];
        
        # Objective
        maximize obj: sum{i in I_2} p[i];
    """)
    
    ampl.set['I_1'] = I[1]
    ampl.set['I_2'] = I[2]
    ampl.set['I_2_p'] = I_2_p
    ampl.set['T'] = T
    
    ampl.param['A_pp_T'] = A_pp_T
    ampl.param['A_p_T'] = A_p_T

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"

    p = ampl.getVariable('p').getValues().toPandas()
    
    # close AMPL
    ampl.close()
    
    return p

In [778]:
def modified_dresher_procedure(U_df, I):
    t = 0
    A_p_T_list = []
    A_p_T = []
    I_2_p = []
    A_T = U_df.transpose()
    
    while True:
        T = range(t)  
        
        z, x = P(T, A_T, A_p_T, I, I_2_p)
        
        # print(f"\nResults P_{t}:")
        # print("z:", z)
        # print("x:\n",x.iloc[:,0])
        
        A_pp_T = A_T - z
        
        p = Q(T, A_p_T, A_pp_T, I, I_2_p)
        
        # print(f"\nResults Q_{t}:")
        # print("p:", p)
        
        t+=1
        
        A_p_T_list.append(A_T.loc[p.iloc[:,0] == 0])
        A_p_T = pd.concat(A_p_T_list, keys=range(t))    
        A_T = A_T.loc[p.iloc[:,0] != 0]
        
        I_2_p = [I[2][i] for i in range(len(I[2])) if p.iloc[i,0] == 0]
        I[2] = [I[2][i] for i in range(len(I[2])) if p.iloc[i,0] != 0]
        
        # Stop condition
        if all(p.iloc[:,0] == 0):
            break
    return x.iloc[:,0]

In [779]:
def find_proper_equilibrium(players, actions, utility_matrices):
    U_df = {
        1: pd.DataFrame(utility_matrices[1], index=actions[1], columns=actions[2]),
        2: pd.DataFrame(list(map(list, zip(*utility_matrices[2]))), index=actions[2], columns=actions[1])
    }

    actions_keys = list(actions.keys())

    I = {
        1: actions,
        2: {p: actions[actions_keys[-p]] for p in players}
    }
    
    return {p: modified_dresher_procedure(U_df[p], I[p]) for p in players}

In [780]:
# Game
Players = [1, 2]

A_1 = ["guess_heads_up", "guess_tails_up"]
A_2 = ["hide penny heads up", "hide penny tails up", "tease"]
I = {1: A_1, 2: A_2}

U1 = [[1,0,0], 
    [0,1,0]]

U2 = [[0,1,1], 
    [1,0,1]]

U = {1: U1, 2: U2}

sigmas = find_proper_equilibrium(Players, I, U)



Gurobi 9.5.1: 

optimal solution; objective 0
Gurobi 9.5.1: optimal solution; objective 2
Gurobi 9.5.1: optimal solution; objective 0.5
1 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0
Gurobi 9.5.1: optimal solution; objective 1
1 simplex iterations
Solution determined by presolve;
objective obj = 0.


In [781]:
from colorama import Fore, Style

colors = [Fore.RED, Fore.GREEN, Fore.YELLOW, Fore.BLUE, Fore.MAGENTA, Fore.CYAN]
for i, p in enumerate(Players):
    print(colors[i % len(colors)] + f"Player {p} strategy is:")
    print(sigmas[p])
    print("")
print(Style.RESET_ALL)

Player 1 strategy is:
guess_heads_up    0.5
guess_tails_up    0.5
Name: x.val, dtype: float64

Player 2 strategy is:
hide penny heads up    0
hide penny tails up    0
tease                  1
Name: x.val, dtype: int64


